In [1]:
import optimization_methods
import plot_methods
import numpy as np
import pandas as pd
import random
import itertools
import seaborn as sns
import math
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Functions initializations and plottings

In [2]:
bukin_function = lambda x, y: 100*math.sqrt(abs(y - 0.01*pow(x, 2))) + 0.01*abs(x+10)
makkormik_function = lambda x, y: math.sin(x + y) + pow(x-y, 2) - 1.5*x + 2.5*y + 1
but_function = lambda x, y: pow(x + 2*y - 7, 2) + pow(2*x + y - 5, 2)

In [3]:
plot_methods.plot_function(func=bukin_function, x_range=np.arange(-15, -4.9, 0.1),
                           y_range=np.arange(-3, 3.1, 0.1),
                           title = 'Bukin Function', only_save=True)

In [4]:
plot_methods.plot_function(func=but_function, x_range=np.arange(-10, 10, 0.1),
                           y_range=np.arange(-10, 10, 0.1),
                           title = 'But Function', only_save=True)

In [5]:
plot_methods.plot_function(func=makkormik_function, x_range=np.arange(-1.5, 4, 0.1),
                           y_range=np.arange(-3, 4, 0.1),
                           title = 'Makkormik Function', only_save=True)

## Genetic Algorithm implementation

In [6]:
class GeneticAlgorithm:

    def __init__(self, fitness_function, lower_bounds, upper_bounds):
        self.fitness_function = fitness_function
        self.lower_bounds = lower_bounds
        self.upper_bounds = upper_bounds

    def _calculate_fitness(self, population_result):
        """define the value of fitness function for an individual"""
        fitness_value = self.fitness_function(*population_result)
        return fitness_value

    def _create_individual(self, qty_variables):
        """creates an individual of population"""
        return [self.lower_bounds[i] + random.random() * (self.upper_bounds[i] - self.lower_bounds[i])
                for i in range(qty_variables)]

    def _create_population(self, qty_variables, qty_individuals):
        """creates a population"""
        return [self._create_individual(qty_variables) for i in range(qty_individuals)]

    def _evaluate_population(self, population):
        """evaluates a fitness of whole population"""
        return np.mean([self._calculate_fitness(single_result)
                         for single_result in population])

    def _mutate(self, individual, mutation_probability):
        """mutates an individual"""
        new_individual = []
        if mutation_probability > random.random():
            for i, ind_value in enumerate(individual):
                randomness = np.random.normal(size=1)
                new_value = ind_value + randomness
                while new_value > self.upper_bounds[i] or new_value < self.lower_bounds[i]:
                    randomness = np.random.normal(size=1)
                    new_value = ind_value + randomness
                new_individual.append(new_value)
            return new_individual
        else:
            return individual

    def _get_best_individual(self, population, criterion_function='min'):
        """selects best individual in population, works with 'min' and 'max' criterion functions"""
        population_fitness_results = np.array([self._calculate_fitness(individual) for individual in population])
        if criterion_function == 'min':
            return population_fitness_results.min(), population[population_fitness_results.argmin()]
        else:
            return population_fitness_results.max(), population[population_fitness_results.argmax()]

    def _select_population(self, population, criterion='min', mutation_probability=0.5):
        """selects best individuals for population"""
        new_population = []
        population_avg_score = self._evaluate_population(population)
        for individual in population:
            if criterion == 'min':
                if self._calculate_fitness(individual) < population_avg_score:
                    new_population.append(individual)
                else:
                    new_population.append(self._mutate(individual, mutation_probability))
            else:
                if self._calculate_fitness(individual) > population_avg_score:
                    new_population.append(individual)
                else:
                    new_population.append(self._mutate(individual, mutation_probability))
        return new_population

    def main(self, population_qty, qty_variables_per_individual, max_iterations=1000,
             criterion_function='min', mutation_probability=0.5):
        """main function"""
        population = self._create_population(qty_variables_per_individual, population_qty)
        best_fitness_function_value, best_solution = self._get_best_individual(population, criterion_function)
        iteration_number = 1
        while iteration_number < max_iterations:
            population = self._select_population(population, criterion_function, mutation_probability)
            best_fitness_function_value, best_solution = self._get_best_individual(population, 
                                                                                        criterion_function)
            iteration_number += 1

        return best_fitness_function_value, best_solution

## Testing according to the variant

In [7]:
alg_bukin = GeneticAlgorithm(bukin_function, [-15, -3], [-5, 3])

In [8]:
bukin_function_result = alg_bukin.main(100, 2, 40000)

In [9]:
alg_but = GeneticAlgorithm(but_function, [-10, -10], [10, 10])

In [10]:
but_function_result = alg_but.main(100, 2, 20000)

In [11]:
alg_makkormik = GeneticAlgorithm(makkormik_function, [-1.5, -3], [4, 4])

In [12]:
makkormik_function_result = alg_makkormik.main(100, 2, 10000)

In [13]:
bukin_function_result

(0.5549905007220453, [array([-12.3654868]), array([1.52908087])])

In [14]:
but_function_result

(0.0078082875801088894, [array([0.97189307]), array([2.9867468])])

In [15]:
makkormik_function_result

(-1.9127523225739056, [array([-0.52791212]), array([-1.53594977])])

## Experiments with different population size and qty of iterations

In [16]:
def make_analysis(function, lower_bounds, upper_bounds, 
                  qty_individuals_list, qty_iterations_list,
                  function_title, qty_variables=2):
    alg = GeneticAlgorithm(function, lower_bounds, upper_bounds)
    df_result = pd.DataFrame(columns = ['{} value'.format(function_title), 'X', 'Y'],
                            index = pd.MultiIndex.from_product([qty_iterations_list, qty_individuals_list]))
    
    df_result.index.names = ['Qty Iterations', 'Qty individuals']
    
    for qty_iter, qty_individ in itertools.product(qty_iterations_list, qty_individuals_list):
        func_value, extremums = alg.main(qty_individ, qty_variables, qty_iter)
        try:
            extremums = list(itertools.chain(*extremums))
        except:
            pass
        df_result.loc[(qty_iter, qty_individ), 
                      '{} value'.format(function_title)] = func_value
        df_result.loc[(qty_iter, qty_individ), 'X'] = extremums[0]
        df_result.loc[(qty_iter, qty_individ), 'Y'] = extremums[1]
    
    for col in df_result.columns:
        df_result[col] = df_result[col].apply(lambda x: x[0] if isinstance(x, type(np.array([]))) else x)
   
    df_result = df_result.style.highlight_min(subset=['{} value'.format(function_title)])

    return df_result

### Bukin function

In [20]:
bukin_result = make_analysis(bukin_function, [-15, -3], [-5, 3], [50, 100, 200],
                             [5000, 10000, 20000], 'Bukin Function')

In [21]:
bukin_result

### But Function

In [22]:
but_result = make_analysis(but_function,  [-10, -10], [10, 10], [50, 100, 200],
                             [5000, 10000, 20000], 'But Function')

In [23]:
but_result

### Makkormik Function

In [24]:
makkormik_result = make_analysis(makkormik_function,  [-1.5, -3], [4, 4], [50, 100, 200],
                             [5000, 10000, 20000], 'Makkormik Function')

In [25]:
makkormik_result

## Comparsion with gradient descent method

### Bukin Function

In [26]:
classic_method_results_bukin = optimization_methods.gradient_descent(bukin_function, [1.5, 0.5], 1e-5)

number of iteration: 2, current point: [ 1.51688065 -0.06529266], function value: 29.83080940613728
number of iteration: 3, current point: [1.51188588 0.09902527], function value: 27.713538939627707
number of iteration: 4, current point: [1.51455588 0.01056346], function value: 11.239595725350021
number of iteration: 5, current point: [1.51414008 0.0242799 ], function value: 3.794405446241665
number of iteration: 6, current point: [1.51421856 0.02168786], function value: 3.6375283832745264
number of iteration: 7, current point: [1.51417755 0.0230416 ], function value: 1.184093155262148
number of iteration: 8, current point: [1.51418177 0.02290207], function value: 0.619084649541567
number of iteration: 9, current point: [1.51418065 0.0229398 ], function value: 0.466843722330998
number of iteration: 10, current point: [1.51418106 0.02292494], function value: 0.2732207238150566
number of iteration: 11, current point: [1.51418016 0.02292865], function value: 0.22638281314664804
number of 

KeyboardInterrupt: 

### But Function

In [27]:
classic_method_results_but = optimization_methods.gradient_descent(but_function, [3, 5], 1e-5)

number of iteration: 2, current point: [0.75 2.75], function value: 1.1249999992333812
number of iteration: 3, current point: [1.03125 3.03125], function value: 0.017578124985490273
number of iteration: 4, current point: [0.99609375 2.99609375], function value: 0.00027465820293115506
number of iteration: 5, current point: [1.00048828 3.00048828], function value: 4.291534420734246e-06
number of iteration: 6, current point: [0.99993896 2.99993896], function value: 6.705522534076417e-08
number of iteration: 7, current point: [1.00000763 3.00000763], function value: 1.0477378961171527e-09
number of iteration: 8, current point: [0.99999905 2.99999905], function value: 1.637090462683051e-11
number of iteration: 9, current point: [1.00000012 3.00000012], function value: 2.557953855089113e-13
Precision: 3.034573095921973e-06


In [28]:
plot_methods.plot_contour_and_scatter_of_descent(but_function, classic_method_results_but, 
                                                x_range=np.arange(-10, 10, 0.1), y_range=np.arange(-10, 10, 0.1),
                                            title = 'But Function contour and scatter of gradient descent method',
                                                only_save=True)

### Makkormik Function

In [29]:
method_results = optimization_methods.gradient_descent(makkormik_function, [0, 0], 1e-5)

number of iteration: 2, current point: [ 0.25 -1.75], function value: -0.747494986589821
number of iteration: 3, current point: [-1.0353686 -1.0353686], function value: -0.9129795073087283
number of iteration: 4, current point: [-0.04568178 -2.04568178], function value: -0.9132189731070119
number of iteration: 5, current point: [-1.04699562 -1.04699562], function value: -0.9132228843397694
number of iteration: 6, current point: [-0.04717052 -2.04717052], function value: -0.9132229536557674
number of iteration: 7, current point: [-1.04719393 -1.04719393], function value: -0.9132229548806063
number of iteration: 8, current point: [-0.5471955 -1.5471955], function value: -1.9132229549737345
Precision: 5.029445771150593e-06


In [30]:
plot_methods.plot_contour_and_scatter_of_descent(makkormik_function, method_results, 
                                                 x_range=np.arange(-1.5, 4, 0.1), y_range=np.arange(-3, 4, 0.1), 
                                            title = 'Makkormik Function contour and scatter of gradient descent method',
                                                only_save=True)